<a href="https://colab.research.google.com/github/vid1988/stockprediction/blob/master/stockforecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#imports and hardware startup
import os, re, time, json
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
print("Tensorflow version " + tf.__version__)

IS_COLAB_BACKEND = 'COLAB_GPU' in os.environ  # this is always set on Colab, the value is 0 or 1 depending on GPU presence
if IS_COLAB_BACKEND:
  from google.colab import auth
  # Authenticates the Colab machine and also the TPU using your
  # credentials so that they can access your private GCS buckets.
  auth.authenticate_user()


# Detect hardware
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
  tpu = None
  gpus = tf.config.experimental.list_logical_devices("GPU")
    
# Select appropriate distribution strategy
if tpu:
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.experimental.TPUStrategy(tpu, steps_per_run=128) # Going back and forth between TPU and host is expensive. Better to run 128 batches on the TPU before reporting back.
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])  
elif len(gpus) > 1:
  strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
  print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on single GPU ', gpus[0].name)
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on CPU')
print("Number of accelerators: ", strategy.num_replicas_in_sync)


In [0]:
#model preperation (use external files, this is just for example) 

def make_model():
    model = tf.keras.Sequential(
      [
        tf.keras.layers.Reshape(input_shape=(28*28,), target_shape=(28, 28, 1), name="image"),
        tf.keras.layers.ZeroPadding2D((2,2)), #reshape 28 x 28 to 32 x 32
        

        tf.keras.layers.Conv2D(filters=6, kernel_size=5, padding='valid', use_bias=True), # no bias necessary before batch norm
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.AveragePooling2D(), 
       
        tf.keras.layers.Conv2D(filters=16, kernel_size=5, padding='valid', use_bias=True),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.AveragePooling2D(),
        tf.keras.layers.Flatten(),
       
        tf.keras.layers.Dense(120, use_bias=True),
        tf.keras.layers.Activation('tanh'),

        tf.keras.layers.Dense(84, use_bias=True),
        tf.keras.layers.Activation('tanh'),
       
        tf.keras.layers.Dense(10, use_bias=True),
        tf.keras.layers.Activation('softmax')

      ])

    model.compile(optimizer='adam', # learning rate will be set by LearningRateScheduler
                  loss='categorical_crossentropy',
                  metrics=['accuracy', tf.keras.metrics.CategoricalCrossentropy()])
    return model
    
with strategy.scope():
    model = make_model()

# print model layers
model.summary()

# set up learning rate decay
lr_decay = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: LEARNING_RATE * LEARNING_RATE_EXP_DECAY**epoch,
    verbose=True)

In [0]:
# training preferences and training start
EPOCHS = 12
steps_per_epoch = 60000//BATCH_SIZE  # 60,000 items in this dataset
#steps_per_epoch=600
print("Steps per epoch: ", steps_per_epoch)
  
# Little wrinkle: in the present version of Tensorfow (1.14), switching a TPU
# between training and evaluation is slow (approx. 10 sec). For small models,
# it is recommeneded to run a single eval at the end.
history = model.fit(training_dataset,
                    steps_per_epoch=steps_per_epoch, epochs=EPOCHS)#,validation_data=validation_dataset)

final_stats = model.evaluate(validation_dataset, steps=1)
print("Validation accuracy: ", final_stats)

In [0]:
# model save
model.save('model.h5') #can be downloaded locally on the left side